# [Lab0] 환경 설정 및 초기화

이 노트북에서는 SageMaker 엔드투엔드 머신러닝 워크플로우를 위한 환경을 설정합니다.

## 주요 설정 내용
- AWS 환경 및 권한 설정
- MLflow 추적 서버 연결
- 데이터 준비 및 S3 업로드
- 실험 환경 초기화

## 1. 필수 라이브러리 및 AWS 환경 설정

In [ ]:
# 필요한 패키지 설치 (주석 해제하여 실행)
# %pip install -q mlflow==2.13.2 sagemaker-mlflow

In [ ]:
# 패키지 설치 후 커널 재시작 (필요시 주석 해제)
# import IPython
# IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
# 필수 라이브러리 임포트
import json
import os
import logging
from time import gmtime, strftime

import boto3
import pandas as pd
import numpy as np
import sagemaker
import mlflow

from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker_studio import Project

print("✅ 라이브러리 임포트 완료")

In [ ]:
# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ 로깅 설정 완료")

In [ ]:
# AWS 세션 및 SageMaker 설정
boto_session = boto3.Session()
sm_client = boto_session.client("sagemaker")
region = boto_session.region_name
sess = sagemaker.Session()

print(f"✅ AWS 리전: {region}")
print(f"✅ SageMaker 세션 초기화 완료")

## 2. SageMaker AI 프로젝트 설정

SageMaker AI 프로젝트를 통해 통합된 리소스 관리를 수행합니다.

In [ ]:
# SageMaker AI 프로젝트 초기화
project = Project()

# MLflow 추적 서버 ARN 가져오기
mlflow_arn = project.mlflow_tracking_server_arn
mlflow_name = mlflow_arn.split('tracking-server/')[-1]

print(f"✅ MLflow 추적 서버 ARN: {mlflow_arn}")
print(f"✅ MLflow 서버 이름: {mlflow_name}")

# MLflow 추적 URI 설정
mlflow.set_tracking_uri(mlflow_arn)

In [ ]:
# S3 버킷 및 IAM 역할 설정
bucket_root = project.s3.root
role = project.iam_role

# S3 URI 파싱
s3_parts = bucket_root.replace("s3://", "").split("/")
bucket = s3_parts[0]
prefix = "/".join(s3_parts[1:] + ["sagemaker/DEMO-xgboost-dm"])

print(f"✅ S3 버킷: {bucket}")
print(f"✅ S3 프리픽스: {prefix}")
print(f"✅ IAM 역할: {role}")

In [ ]:
# 프로젝트 데이터베이스 정보 가져오기
catalog = project.connection().catalog()
project_database = catalog.databases[0].name

print(f"✅ 프로젝트 데이터베이스: {project_database}")

## 3. 실험 설정

MLflow를 사용한 실험 추적을 위한 설정을 수행합니다.

In [ ]:
# 실험 이름 생성
experiment_suffix = strftime('%d-%H-%M-%S', gmtime())
experiment_name = f"end-to-end-experiment-{experiment_suffix}"

print(f"✅ 실험 이름: {experiment_name}")

## 4. 데이터 준비

은행 마케팅 데이터셋을 준비하고 기본적인 탐색을 수행합니다.

In [ ]:
import zipfile

# 데이터 압축 해제
with zipfile.ZipFile('bank-additional.zip', 'r') as zip_ref:
    zip_ref.extractall('.')
    
print("✅ 데이터 압축 해제 완료")

In [ ]:
# 데이터 로드 및 기본 정보 확인
data = pd.read_csv('./bank-additional/bank-additional-full.csv')

print(f"✅ 데이터 로드 완료")
print(f"   - 데이터 크기: {data.shape}")
print(f"   - 컬럼 수: {len(data.columns)}")
print(f"   - 타겟 변수 분포:")
print(data['y'].value_counts())

# 처음 5행 표시
print("\n📊 데이터 미리보기:")
display(data.head())

## 5. 변수 저장

다음 노트북에서 사용할 중요한 변수들을 저장합니다.

In [ ]:
# 중요한 변수들을 다음 노트북에서 사용하기 위해 저장
%store bucket
%store prefix
%store role
%store region
%store mlflow_arn
%store mlflow_name
%store experiment_name

print("✅ 모든 변수가 저장되었습니다.")
print("\n📋 저장된 변수 요약:")
print(f"   - S3 버킷: {bucket}")
print(f"   - S3 프리픽스: {prefix}")
print(f"   - 리전: {region}")
print(f"   - 실험 이름: {experiment_name}")
print(f"   - MLflow 서버: {mlflow_name}")

## ✅ 설정 완료

환경 설정이 완료되었습니다! 이제 다음 노트북 `1-preprocessing.ipynb`로 진행하여 데이터 전처리를 수행할 수 있습니다.

### 다음 단계
1. **데이터 전처리** (`1-preprocessing.ipynb`): SageMaker Processing을 사용한 데이터 전처리
2. **모델 훈련** (`2-training.ipynb`): SageMaker Training Job을 사용한 모델 훈련
3. **모델 배포** (`3-test-and-deploy.ipynb`): 모델 평가 및 배포
4. **파이프라인 구축** (`4-pipelines.ipynb`): 엔드투엔드 ML 파이프라인 구축